# Single Image Super-Resolution
This is the primary training notebook for training models for all the loss functions

## Importing required libraries

In [1]:
import os
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import io, transforms
import torchvision.transforms.functional as F
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from collections import OrderedDict
import random
from preprocess import *

In [2]:
!pip install torchsummary
!pip install pytorch_msssim
!pip install torchmetrics

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import sys
sys.path.append('/home/ec2-user/.local/lib/python3.7/site-packages')

In [4]:
# from pytorch_msssim import ssim
from torchsummary import summary
from torchmetrics.functional import structural_similarity_index_measure as ssim

In [5]:
print(torch.__version__)
print(torchvision.__version__)
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0))

1.13.0+cu117
0.14.0+cu117
Tesla T4
_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Data Pre-processing and Loading

In [7]:
# Paired data transformation for training including 0 degrees
paired_transform = PairedTransform([
    RandomHorizontalFlip(),
    RandomVerticalFlip(),
    RandomRotationSpecific([0, 90, 180, 270])
])

tensor_transform = transforms.Compose([
    transforms.ToTensor(),  # Convert the image to a tensor with values between [0, 1]
    # transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to [-1, 1]
])

# Initialize dataset
batch_size = 16
hr_dir = "HR_patches"
# lr_x2_dir = "LR_x2_patches"
lr_x4_dir = "LR_x4_patches"

# Initialize the dataset
dataset = SRDataset(hr_dir, 
                    lr_x4_dir,
                    paired_transform=paired_transform, 
                    tensor_transform=tensor_transform,
                    paired_transform_prob=0.5)

# Split the dataset into a smaller subset
subset_size = int(0.35 * len(dataset))
discard_size = len(dataset) - subset_size
subset_dataset, _ = random_split(dataset, [subset_size, discard_size])

# Second split: 90% for training, 10% for validation
train_size = int(0.9 * len(subset_dataset))
val_size = len(subset_dataset) - train_size
train_dataset, val_dataset = random_split(subset_dataset, [train_size, val_size])

# Create subsets for training and validation
train_loader = DataLoader(train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True, 
                          num_workers=4,
                          pin_memory=True,
                          collate_fn=custom_collate_fn)

val_loader = DataLoader(val_dataset, 
                        batch_size=batch_size, 
                        shuffle=False, 
                        num_workers=4,
                        pin_memory=True,
                        collate_fn=custom_collate_fn)

In [8]:
len(train_dataset), len(val_dataset)

(18900, 2100)

## Defining Model Architecture

In [9]:
# Defining Inception Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, batch_norm=False):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0, bias=False)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels) if batch_norm else None

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.relu(out)
        out = self.conv2(out)
        if self.bn1:
            out = self.bn1(out)
        out = self.relu(out)
        out = out + residual
        out = self.relu(out)
        return out

# Defining Residual-in-Residual Block
class ResidualInResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, batch_norm=False):
        super(ResidualInResidualBlock, self).__init__()
        self.res1 = ResidualBlock(in_channels, out_channels, batch_norm)
        self.res2 = ResidualBlock(out_channels, out_channels, batch_norm)
        # self.res3 = ResidualBlock(out_channels, out_channels, batch_norm)

    def forward(self, x):
        out = self.res1(x)
        out = self.res2(out)
        # out = self.res3(out)
        return out + x

# Defining the U-Net architecture
class UNetSRx4RiR(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, num_features=64, dropout_rate=0.0):
        super(UNetSRx4RiR, self).__init__()

        # Encoder
        self.conv1 = nn.Conv2d(in_channels, num_features, kernel_size=3, padding=1)
        self.encoder1 = ResidualInResidualBlock(num_features, num_features, batch_norm=True)
        self.downsample1 = self.downsample(num_features, num_features * 2, dropout_rate)

        self.encoder2 = ResidualInResidualBlock(num_features * 2, num_features * 2, batch_norm=True)
        self.downsample2 = self.downsample(num_features * 2, num_features * 4, dropout_rate)

        self.encoder3 = ResidualInResidualBlock(num_features * 4, num_features * 4, batch_norm=True)
        self.downsample3 = self.downsample(num_features * 4, num_features * 8, dropout_rate)

        self.encoder4 = ResidualInResidualBlock(num_features * 8, num_features * 8, batch_norm=True)
        self.downsample4 = self.downsample(num_features * 8, num_features * 16, dropout_rate)

        # Bottleneck
        self.bottleneck = ResidualInResidualBlock(num_features * 16, num_features * 16, batch_norm=True)

        # Decoder
        
        self.upconv4 = nn.ConvTranspose2d(num_features * 16, num_features * 8, kernel_size=2, stride=2)
        self.decoder4 = ResidualInResidualBlock(num_features * 16, num_features * 16, batch_norm=True)
        self.conv3 = nn.Conv2d(num_features * 16, num_features * 8, kernel_size=3, padding=1)
        
        self.upconv3 = nn.ConvTranspose2d(num_features * 8, num_features * 4, kernel_size=2, stride=2)
        self.decoder3 = ResidualInResidualBlock(num_features * 8, num_features * 8, batch_norm=True)
        self.conv4 = nn.Conv2d(num_features * 8, num_features * 4, kernel_size=3, padding=1)

        self.upconv2 = nn.ConvTranspose2d(num_features * 4, num_features * 2, kernel_size=2, stride=2)
        self.decoder2 = ResidualInResidualBlock(num_features * 4, num_features * 4, batch_norm=True)
        self.conv5 = nn.Conv2d(num_features * 4, num_features * 2, kernel_size=3, padding=1)

        self.upconv1 = nn.ConvTranspose2d(num_features * 2, num_features, kernel_size=2, stride=2)
        self.decoder1 = ResidualInResidualBlock(num_features * 2, num_features * 2, batch_norm=True)

        # Final output layer
        self.final_conv = nn.Sequential(
            nn.Conv2d(num_features * 2, num_features, kernel_size=3, padding=1),
            nn.Conv2d(num_features, out_channels, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def downsample(self, in_channels, out_channels, dropout_rate, batch_norm=True):
        layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=2)]
        if batch_norm:
            layers.append(nn.BatchNorm2d(out_channels))
        layers.append(nn.ReLU())
        if dropout_rate > 0:
            layers.append(nn.Dropout(dropout_rate))
        return nn.Sequential(*layers)

    def forward(self, x):
        # First upscale x4 using bicubic interpolation
        x = torch.nn.functional.interpolate(x, scale_factor=4, mode='bicubic', align_corners=False)

        # Encoder
        enc1 = self.conv1(x) 
        enc1 = self.encoder1(enc1) 

        enc2 = self.downsample1(enc1) 
        enc2 = self.encoder2(enc2)

        enc3 = self.downsample2(enc2)
        enc3 = self.encoder3(enc3) 

        enc4 = self.downsample3(enc3)
        enc4 = self.encoder4(enc4)

        # Bottleneck
        bottleneck = self.downsample4(enc4)
        bottleneck = self.bottleneck(bottleneck)

        # Decoder
        dec4 = self.upconv4(bottleneck) 
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        dec4 = self.conv3(dec4)
        
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec3 = self.conv4(dec3)

        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec2 = self.conv5(dec2)

        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)

        sr = self.final_conv(dec1)

        return sr

In [10]:
model_1 = UNetSRx4RiR(3, 3, 64, 0.0)
model_1.to(device)
summary(model_1, input_size=(3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
            Conv2d-2         [-1, 64, 256, 256]           4,096
              ReLU-3         [-1, 64, 256, 256]               0
            Conv2d-4         [-1, 64, 256, 256]          36,864
       BatchNorm2d-5         [-1, 64, 256, 256]             128
              ReLU-6         [-1, 64, 256, 256]               0
              ReLU-7         [-1, 64, 256, 256]               0
     ResidualBlock-8         [-1, 64, 256, 256]               0
            Conv2d-9         [-1, 64, 256, 256]           4,096
             ReLU-10         [-1, 64, 256, 256]               0
           Conv2d-11         [-1, 64, 256, 256]          36,864
      BatchNorm2d-12         [-1, 64, 256, 256]             128
             ReLU-13         [-1, 64, 256, 256]               0
             ReLU-14         [-1, 64, 2

## Defining Loss Functions and Evaluation Metrics

### Defining PSNR metric

In [11]:
def calculate_psnr(img1, img2, max_pixel_value=1.0):
    mse = torch.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    psnr = 10 * torch.log10((max_pixel_value ** 2) / mse)
    return psnr

### Defining Tukey Loss

In [11]:
class TukeyLoss(nn.Module):
    def __init__(self, c=0.3):  # Adjusted c value for normalized data
        super(TukeyLoss, self).__init__()
        self.c = c

    def forward(self, input, target):
        error = input - target
        abs_error = torch.abs(error)

        # Tukey loss calculation
        mask = abs_error <= self.c
        tukey_loss = torch.where(
            mask,
            (self.c ** 2 / 6) * (1 - (1 - (error / self.c) ** 2) ** 3),
            (self.c ** 2 / 6) * torch.ones_like(error)
        )

        return tukey_loss.mean()

### Defining Charbonnier loss

In [12]:
class CharbonnierLoss(nn.Module):
    def __init__(self, epsilon=1e-3):
        super(CharbonnierLoss, self).__init__()
        self.epsilon = epsilon

    def forward(self, x, y):
        diff = x - y
        loss = torch.mean(torch.sqrt(diff * diff + self.epsilon ** 2))
        return loss

### Defining Total Variation (TV) loss and combined TV + MAE loss

In [13]:
class TVLoss(nn.Module):
    def __init__(self, tv_weight=1e-4):
        super(TVLoss, self).__init__()
        self.tv_weight = tv_weight

    def forward(self, x):
        batch_size = x.size(0)
        h_x = x.size(2)
        w_x = x.size(3)
        count_h = self._tensor_size(x[:, :, 1:, :])
        count_w = self._tensor_size(x[:, :, :, 1:])
        h_tv = torch.pow(x[:, :, 1:, :] - x[:, :, :-1, :], 2).sum()
        w_tv = torch.pow(x[:, :, :, 1:] - x[:, :, :, :-1], 2).sum()
        return self.tv_weight * 2 * (h_tv / count_h + w_tv / count_w) / batch_size

    @staticmethod
    def _tensor_size(t):
        return t.size(1) * t.size(2) * t.size(3)
    
def combined_tv_mae_loss(output, target):
    """Calculate combined MAE and TV loss."""
    mae_loss = l1_loss(output, target)
    tv_out = tv_loss(output)
    return tv_out + mae_loss

### Defining combined SSIM loss + MAE loss

In [12]:
def combined_ssim_l1_loss(output, target, alpha=0.7):
    ssim_out = 1 - ssim(output, target, data_range=1.0)
    mae_loss = torch.nn.functional.l1_loss(output, target)
    return alpha * mae_loss + (1 - alpha) * ssim_out + 1e-8

### Defining Early Stopping class

In [12]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.delta = delta

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

## Model Training

### Training the model for MAE loss

In [ ]:
from torch.cuda.amp import GradScaler, autocast

torch.autograd.set_detect_anomaly(True)

if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = UNetSRx4RiR(in_channels=3, out_channels=3, num_features=64, dropout_rate=0.0).to(device)
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=0.00005)
    scaler = GradScaler()
    
    # Initialize TensorBoard writer
    from torch.utils.tensorboard import SummaryWriter
    writer = SummaryWriter('runs/UNetSRx4_L1_fin')

    # Initialize early stopping
    early_stopping = EarlyStopping(patience=5, verbose=True)
    
    # Initialize learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=2, verbose=True)

    num_epochs = 25
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_psnr = 0.0
        running_ssim = 0.0

        # Wrap the train_loader with tqdm for progress bar
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
        
        for i, (hr_patches, lr_patches) in enumerate(progress_bar):
            hr_patches, lr_patches = hr_patches.to(device), lr_patches.to(device)
            
            optimizer.zero_grad()
            with autocast():
                outputs = model(lr_patches)
                loss = criterion(outputs, hr_patches)
                if torch.isnan(loss).any():
                    print("Loss is NaN")
                    break
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            running_loss += loss.item()
            running_psnr += calculate_psnr(outputs, hr_patches).item()
            outputs = outputs.float()
            hr_patches = hr_patches.float()
            running_ssim += ssim(outputs, hr_patches, data_range=1.0).item()

            # Update the progress bar with the running loss
            progress_bar.set_postfix(loss=loss.item())
        
        avg_train_loss = running_loss / len(train_loader)
        avg_train_psnr = running_psnr / len(train_loader)
        avg_train_ssim = running_ssim / len(train_loader)
        writer.add_scalar('Loss/train', avg_train_loss, epoch)
        writer.add_scalar('PSNR/train', avg_train_psnr, epoch)
        writer.add_scalar('SSIM/train', avg_train_ssim, epoch)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train PSNR: {avg_train_psnr:.4f}, Train SSIM: {avg_train_ssim:.4f}')
    
        # Validation loop
        model.eval()
        val_loss = 0.0
        val_psnr = 0.0
        val_ssim = 0.0     
        with torch.no_grad():
            for hr_patches, lr_patches in val_loader:
                hr_patches, lr_patches = hr_patches.to(device), lr_patches.to(device)
                with autocast(enabled=True):
                    outputs = model(lr_patches)
                    loss = criterion(outputs, hr_patches)
                    if torch.isnan(loss).any():
                        print("Loss is NaN")
                        break
                val_loss += loss.item()
                val_psnr += calculate_psnr(outputs, hr_patches).item()
                # Convert to float32 before calculating SSIM
                outputs = outputs.float()
                hr_patches = hr_patches.float()
                val_ssim += ssim(outputs, hr_patches, data_range=1.0).item()
    
        avg_val_loss = val_loss / len(val_loader)
        avg_val_psnr = val_psnr / len(val_loader)
        avg_val_ssim = val_ssim / len(val_loader)
        writer.add_scalar('Loss/val', avg_val_loss, epoch)
        writer.add_scalar('PSNR/val', avg_val_psnr, epoch)
        writer.add_scalar('SSIM/val', avg_val_ssim, epoch)
        print(f'Validation Loss: {avg_val_loss:.4f}, Validation PSNR: {avg_val_psnr:.4f}, Validation SSIM: {avg_val_ssim:.4f}')

        # Check early stopping condition
        early_stopping(avg_val_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
        # Step the scheduler
        scheduler.step(avg_val_loss)
    
    print('Training complete.')

2024-09-07 04:55:39.783390: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-07 04:55:39.938763: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-07 04:55:40.670297: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-09-07 04:55:40.670367: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

Epoch [1/25], Train Loss: 0.0471, Train PSNR: 23.4607, Train SSIM: 0.7081


Validation Loss: 0.0338, Validation PSNR: 25.0414, Validation SSIM: 0.7598
Validation loss decreased (inf --> 0.033751).  Saving model ...


Epoch 2/25: 100%|██████████| 1182/1182 [23:23<00:00,  1.19s/batch, loss=0.0268]

Epoch [2/25], Train Loss: 0.0350, Train PSNR: 24.9670, Train SSIM: 0.7618


...

Validation Loss: 0.0259, Validation PSNR: 26.2640, Validation SSIM: 0.8089
Validation loss decreased (0.025996 --> 0.025868).  Saving model ...


Epoch 24/25: 100%|██████████| 1182/1182 [23:21<00:00,  1.19s/batch, loss=0.0217]

Epoch [24/25], Train Loss: 0.0258, Train PSNR: 26.3511, Train SSIM: 0.8098


Validation Loss: 0.0258, Validation PSNR: 26.2700, Validation SSIM: 0.8102
Validation loss decreased (0.025868 --> 0.025831).  Saving model ...


Epoch 25/25: 100%|█████████▉| 1180/1182 [23:21<00:02,  1.19s/batch, loss=0.0265]

In [ ]:
torch.save(model, "UNet_L1_final.pth")

### Training the model for MSE loss

In [14]:
from torch.cuda.amp import GradScaler, autocast

torch.autograd.set_detect_anomaly(True)

if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = UNetSRx4RiR(in_channels=3, out_channels=3, num_features=64, dropout_rate=0.0).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    scaler = GradScaler()
    
    # Initialize TensorBoard writer
    from torch.utils.tensorboard import SummaryWriter
    writer = SummaryWriter('runs/UNetSRx4_L2')

    # Initialize early stopping
    early_stopping = EarlyStopping(patience=5, verbose=True)
    
    # Initialize learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=2, verbose=True)

    num_epochs = 25
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_psnr = 0.0
        running_ssim = 0.0

        # Wrap the train_loader with tqdm for progress bar
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
        
        for i, (hr_patches, lr_patches) in enumerate(progress_bar):
            hr_patches, lr_patches = hr_patches.to(device), lr_patches.to(device)
            
            optimizer.zero_grad()
            with autocast():
                outputs = model(lr_patches)
                loss = criterion(outputs, hr_patches)
                if torch.isnan(loss).any():
                    print("Loss is NaN")
                    break
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            running_loss += loss.item()
            running_psnr += calculate_psnr(outputs, hr_patches).item()
            outputs = outputs.float()
            hr_patches = hr_patches.float()
            running_ssim += ssim(outputs, hr_patches, data_range=1.0).item()

            # Update the progress bar with the running loss
            progress_bar.set_postfix(loss=loss.item())
        
        avg_train_loss = running_loss / len(train_loader)
        avg_train_psnr = running_psnr / len(train_loader)
        avg_train_ssim = running_ssim / len(train_loader)
        writer.add_scalar('Loss/train', avg_train_loss, epoch)
        writer.add_scalar('PSNR/train', avg_train_psnr, epoch)
        writer.add_scalar('SSIM/train', avg_train_ssim, epoch)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train PSNR: {avg_train_psnr:.4f}, Train SSIM: {avg_train_ssim:.4f}')
    
        # Validation loop
        model.eval()
        val_loss = 0.0
        val_psnr = 0.0
        val_ssim = 0.0     
        with torch.no_grad():
            for hr_patches, lr_patches in val_loader:
                hr_patches, lr_patches = hr_patches.to(device), lr_patches.to(device)
                with autocast(enabled=True):
                    outputs = model(lr_patches)
                    loss = criterion(outputs, hr_patches)
                    if torch.isnan(loss).any():
                        print("Loss is NaN")
                        break
                val_loss += loss.item()
                val_psnr += calculate_psnr(outputs, hr_patches).item()
                # Convert to float32 before calculating SSIM
                outputs = outputs.float()
                hr_patches = hr_patches.float()
                val_ssim += ssim(outputs, hr_patches, data_range=1.0).item()
    
        avg_val_loss = val_loss / len(val_loader)
        avg_val_psnr = val_psnr / len(val_loader)
        avg_val_ssim = val_ssim / len(val_loader)
        writer.add_scalar('Loss/val', avg_val_loss, epoch)
        writer.add_scalar('PSNR/val', avg_val_psnr, epoch)
        writer.add_scalar('SSIM/val', avg_val_ssim, epoch)
        print(f'Validation Loss: {avg_val_loss:.4f}, Validation PSNR: {avg_val_psnr:.4f}, Validation SSIM: {avg_val_ssim:.4f}')

        # Check early stopping condition
        early_stopping(avg_val_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
        # Step the scheduler
        scheduler.step(avg_val_loss)
    
    print('Training complete.')

Epoch 1/25: 100%|██████████| 1182/1182 [25:41<00:00,  1.30s/batch, loss=0.00643]

Epoch [1/25], Train Loss: 0.0048, Train PSNR: 23.6951, Train SSIM: 0.7243


Validation Loss: 0.0032, Validation PSNR: 25.1840, Validation SSIM: 0.7579
Validation loss decreased (inf --> 0.003196).  Saving model ...


Epoch 2/25: 100%|██████████| 1182/1182 [23:18<00:00,  1.18s/batch, loss=0.008]  

Epoch [2/25], Train Loss: 0.0032, Train PSNR: 25.1770, Train SSIM: 0.7694


...

Epoch [24/25], Train Loss: 0.0024, Train PSNR: 26.4427, Train SSIM: 0.8101


Validation Loss: 0.0027, Validation PSNR: 26.0393, Validation SSIM: 0.8028
EarlyStopping counter: 3 out of 5
Epoch 00024: reducing learning rate of group 0 to 2.5000e-05.


Epoch 25/25: 100%|██████████| 1182/1182 [23:32<00:00,  1.20s/batch, loss=0.00137] 

Epoch [25/25], Train Loss: 0.0024, Train PSNR: 26.5032, Train SSIM: 0.8119


Validation Loss: 0.0025, Validation PSNR: 26.3680, Validation SSIM: 0.8077
Validation loss decreased (0.002501 --> 0.002475).  Saving model ...
Training complete.


In [17]:
torch.save(model, "UNet_L2_final.pth")

### Training the model for Tukey loss (C = 0.3)

In [13]:
from torch.cuda.amp import GradScaler, autocast

torch.autograd.set_detect_anomaly(True)

if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = UNetSRx4RiR(in_channels=3, out_channels=3, num_features=64, dropout_rate=0.0).to(device)
    criterion = TukeyLoss(c=0.3)
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    scaler = GradScaler()
    
    # Initialize TensorBoard writer
    from torch.utils.tensorboard import SummaryWriter
    writer = SummaryWriter('runs/UNetSRx4_Tukey')

    # Initialize early stopping
    early_stopping = EarlyStopping(patience=5, verbose=True)
    
    # Initialize learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=2, verbose=True)

    num_epochs = 25
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_psnr = 0.0
        running_ssim = 0.0

        # Wrap the train_loader with tqdm for progress bar
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
        
        for i, (hr_patches, lr_patches) in enumerate(progress_bar):
            hr_patches, lr_patches = hr_patches.to(device), lr_patches.to(device)
            
            optimizer.zero_grad()
            with autocast():
                outputs = model(lr_patches)
                loss = criterion(outputs, hr_patches)
                if torch.isnan(loss).any():
                    print("Loss is NaN")
                    break
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            running_loss += loss.item()
            running_psnr += calculate_psnr(outputs, hr_patches).item()
            outputs = outputs.float()
            hr_patches = hr_patches.float()
            running_ssim += ssim(outputs, hr_patches, data_range=1.0).item()

            # Update the progress bar with the running loss
            progress_bar.set_postfix(loss=loss.item())
        
        avg_train_loss = running_loss / len(train_loader)
        avg_train_psnr = running_psnr / len(train_loader)
        avg_train_ssim = running_ssim / len(train_loader)
        writer.add_scalar('Loss/train', avg_train_loss, epoch)
        writer.add_scalar('PSNR/train', avg_train_psnr, epoch)
        writer.add_scalar('SSIM/train', avg_train_ssim, epoch)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train PSNR: {avg_train_psnr:.4f}, Train SSIM: {avg_train_ssim:.4f}')
    
        # Validation loop
        model.eval()
        val_loss = 0.0
        val_psnr = 0.0
        val_ssim = 0.0     
        with torch.no_grad():
            for hr_patches, lr_patches in val_loader:
                hr_patches, lr_patches = hr_patches.to(device), lr_patches.to(device)
                with autocast(enabled=True):
                    outputs = model(lr_patches)
                    loss = criterion(outputs, hr_patches)
                    if torch.isnan(loss).any():
                        print("Loss is NaN")
                        break
                val_loss += loss.item()
                val_psnr += calculate_psnr(outputs, hr_patches).item()
                # Convert to float32 before calculating SSIM
                outputs = outputs.float()
                hr_patches = hr_patches.float()
                val_ssim += ssim(outputs, hr_patches, data_range=1.0).item()
    
        avg_val_loss = val_loss / len(val_loader)
        avg_val_psnr = val_psnr / len(val_loader)
        avg_val_ssim = val_ssim / len(val_loader)
        writer.add_scalar('Loss/val', avg_val_loss, epoch)
        writer.add_scalar('PSNR/val', avg_val_psnr, epoch)
        writer.add_scalar('SSIM/val', avg_val_ssim, epoch)
        print(f'Validation Loss: {avg_val_loss:.4f}, Validation PSNR: {avg_val_psnr:.4f}, Validation SSIM: {avg_val_ssim:.4f}')

        # Check early stopping condition
        early_stopping(avg_val_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
        # Step the scheduler
        scheduler.step(avg_val_loss)
    
    print('Training complete.')

2024-09-01 17:53:37.455563: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-01 17:53:37.595472: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-01 17:53:38.294487: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-09-01 17:53:38.294561: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

Epoch [1/25], Train Loss: 0.0016, Train PSNR: 23.7686, Train SSIM: 0.7276


Validation Loss: 0.0010, Validation PSNR: 25.4096, Validation SSIM: 0.7745
Validation loss decreased (inf --> 0.001032).  Saving model ...


Epoch 2/25: 100%|██████████| 1182/1182 [23:06<00:00,  1.17s/batch, loss=0.000624]

Epoch [2/25], Train Loss: 0.0011, Train PSNR: 25.2389, Train SSIM: 0.7747


...

Epoch [24/25], Train Loss: 0.0008, Train PSNR: 26.4036, Train SSIM: 0.8130


Validation Loss: 0.0008, Validation PSNR: 26.1608, Validation SSIM: 0.8085
EarlyStopping counter: 1 out of 5


Epoch 25/25: 100%|██████████| 1182/1182 [23:21<00:00,  1.19s/batch, loss=0.00037] 

Epoch [25/25], Train Loss: 0.0008, Train PSNR: 26.4178, Train SSIM: 0.8130


Validation Loss: 0.0008, Validation PSNR: 26.4577, Validation SSIM: 0.8140
Validation loss decreased (0.000821 --> 0.000818).  Saving model ...
Training complete.


In [14]:
torch.save(model, "UNet_Tukey_c-0.3_final.pth")

### Training the model for SSIM + MAE loss

In [ ]:
from torch.cuda.amp import GradScaler, autocast

torch.autograd.set_detect_anomaly(True)

if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = UNetSRx4RiR(in_channels=3, out_channels=3, num_features=64, dropout_rate=0.0).to(device)
    criterion = combined_ssim_l1_loss
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    scaler = GradScaler()
    
    # Initialize TensorBoard writer
    from torch.utils.tensorboard import SummaryWriter
    writer = SummaryWriter('runs/UNetSRx4_ssim')

    # Initialize early stopping
    early_stopping = EarlyStopping(patience=5, verbose=True)
    
    # Initialize learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=2, verbose=True)

    num_epochs = 25
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_psnr = 0.0
        running_ssim = 0.0

        # Wrap the train_loader with tqdm for progress bar
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
        
        for i, (hr_patches, lr_patches) in enumerate(progress_bar):
            hr_patches, lr_patches = hr_patches.to(device), lr_patches.to(device)
            
            optimizer.zero_grad()
            with autocast(enabled=False):
                outputs = model(lr_patches)
                loss = criterion(outputs, hr_patches)
                if torch.isnan(loss).any():
                    print("Loss is NaN")
                    break
            
            scaler.scale(loss).backward()
            # Apply gradient clipping
            # nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
            
            running_loss += loss.item()
            running_psnr += calculate_psnr(outputs, hr_patches).item()
            outputs = outputs.float()
            hr_patches = hr_patches.float()
            running_ssim += ssim(outputs, hr_patches, data_range=1.0).item()

            # Update the progress bar with the running loss
            progress_bar.set_postfix(loss=loss.item())
        
        avg_train_loss = running_loss / len(train_loader)
        avg_train_psnr = running_psnr / len(train_loader)
        avg_train_ssim = running_ssim / len(train_loader)
        writer.add_scalar('Loss/train', avg_train_loss, epoch)
        writer.add_scalar('PSNR/train', avg_train_psnr, epoch)
        writer.add_scalar('SSIM/train', avg_train_ssim, epoch)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train PSNR: {avg_train_psnr:.4f}, Train SSIM: {avg_train_ssim:.4f}')
    
        # Validation loop
        model.eval()
        val_loss = 0.0
        val_psnr = 0.0
        val_ssim = 0.0     
        with torch.no_grad():
            for hr_patches, lr_patches in val_loader:
                hr_patches, lr_patches = hr_patches.to(device), lr_patches.to(device)
                with autocast(enabled=False):
                    outputs = model(lr_patches)
                    loss = criterion(outputs, hr_patches)
                    if torch.isnan(loss).any():
                        print("Loss is NaN")
                        break
                val_loss += loss.item()
                val_psnr += calculate_psnr(outputs, hr_patches).item()
                # Convert to float32 before calculating SSIM
                outputs = outputs.float()
                hr_patches = hr_patches.float()
                val_ssim += ssim(outputs, hr_patches, data_range=1.0).item()
    
        avg_val_loss = val_loss / len(val_loader)
        avg_val_psnr = val_psnr / len(val_loader)
        avg_val_ssim = val_ssim / len(val_loader)
        writer.add_scalar('Loss/val', avg_val_loss, epoch)
        writer.add_scalar('PSNR/val', avg_val_psnr, epoch)
        writer.add_scalar('SSIM/val', avg_val_ssim, epoch)
        print(f'Validation Loss: {avg_val_loss:.4f}, Validation PSNR: {avg_val_psnr:.4f}, Validation SSIM: {avg_val_ssim:.4f}')

        # Check early stopping condition
        early_stopping(avg_val_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
        # Step the scheduler
        scheduler.step(avg_val_loss)
    
    print('Training complete.')

2024-09-02 07:38:50.728703: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-02 07:38:50.874078: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-02 07:38:51.596594: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-09-02 07:38:51.596666: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

Epoch [1/25], Train Loss: 0.0985, Train PSNR: 24.3744, Train SSIM: 0.7674


Validation Loss: 0.0856, Validation PSNR: 25.3044, Validation SSIM: 0.7966
Validation loss decreased (inf --> 0.085587).  Saving model ...


Epoch 2/25: 100%|██████████| 2363/2363 [1:07:03<00:00,  1.70s/batch, loss=0.0927]

Epoch [2/25], Train Loss: 0.0837, Train PSNR: 25.5299, Train SSIM: 0.7971


...

Validation Loss: 0.0741, Validation PSNR: 26.3242, Validation SSIM: 0.8197
EarlyStopping counter: 1 out of 5


Epoch 19/25: 100%|██████████| 2363/2363 [1:07:02<00:00,  1.70s/batch, loss=0.0702]

Epoch [19/25], Train Loss: 0.0721, Train PSNR: 26.5160, Train SSIM: 0.8216


Validation Loss: 0.0722, Validation PSNR: 26.5913, Validation SSIM: 0.8206
Validation loss decreased (0.072357 --> 0.072178).  Saving model ...


Epoch 20/25:  76%|███████▌  | 1791/2363 [50:49<16:13,  1.70s/batch, loss=0.0776] 

Continuing the training after epoch 19 as the session abrutly ended

In [13]:
# Continuing the training after epoch 19 as the session abrutly ended
from torch.cuda.amp import GradScaler, autocast

torch.autograd.set_detect_anomaly(True)

if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = UNetSRx4RiR(in_channels=3, out_channels=3, num_features=64, dropout_rate=0.0).to(device)
    criterion = combined_ssim_l1_loss
    
    # Load the model's state dict from the checkpoint
    checkpoint = torch.load('checkpoint.pt')
    model.load_state_dict(checkpoint)
    
    optimizer = optim.Adam(model.parameters(), lr=0.00005) # Reducing the loss function slightly
    scaler = GradScaler()
    start_epoch = 19  # Start from the next epoch
    
    # Initialize TensorBoard writer
    from torch.utils.tensorboard import SummaryWriter
    writer = SummaryWriter('runs/UNetSRx4_ssim')

    # Initialize early stopping
    early_stopping = EarlyStopping(patience=5, verbose=True)
    
    # Initialize learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=2, verbose=True)

    num_epochs = 25
    for epoch in range(start_epoch, num_epochs): # Start from start_rpoch
        model.train()
        running_loss = 0.0
        running_psnr = 0.0
        running_ssim = 0.0

        # Wrap the train_loader with tqdm for progress bar
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
        
        for i, (hr_patches, lr_patches) in enumerate(progress_bar):
            hr_patches, lr_patches = hr_patches.to(device), lr_patches.to(device)
            
            optimizer.zero_grad()
            with autocast(enabled=False):
                outputs = model(lr_patches)
                loss = criterion(outputs, hr_patches)
                if torch.isnan(loss).any():
                    print("Loss is NaN")
                    break
            
            scaler.scale(loss).backward()
            # Apply gradient clipping
            # nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
            
            running_loss += loss.item()
            running_psnr += calculate_psnr(outputs, hr_patches).item()
            outputs = outputs.float()
            hr_patches = hr_patches.float()
            running_ssim += ssim(outputs, hr_patches, data_range=1.0).item()

            # Update the progress bar with the running loss
            progress_bar.set_postfix(loss=loss.item())
        
        avg_train_loss = running_loss / len(train_loader)
        avg_train_psnr = running_psnr / len(train_loader)
        avg_train_ssim = running_ssim / len(train_loader)
        writer.add_scalar('Loss/train', avg_train_loss, epoch)
        writer.add_scalar('PSNR/train', avg_train_psnr, epoch)
        writer.add_scalar('SSIM/train', avg_train_ssim, epoch)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train PSNR: {avg_train_psnr:.4f}, Train SSIM: {avg_train_ssim:.4f}')
    
        # Validation loop
        model.eval()
        val_loss = 0.0
        val_psnr = 0.0
        val_ssim = 0.0     
        with torch.no_grad():
            for hr_patches, lr_patches in val_loader:
                hr_patches, lr_patches = hr_patches.to(device), lr_patches.to(device)
                with autocast(enabled=False):
                    outputs = model(lr_patches)
                    loss = criterion(outputs, hr_patches)
                    if torch.isnan(loss).any():
                        print("Loss is NaN")
                        break
                val_loss += loss.item()
                val_psnr += calculate_psnr(outputs, hr_patches).item()
                # Convert to float32 before calculating SSIM
                outputs = outputs.float()
                hr_patches = hr_patches.float()
                val_ssim += ssim(outputs, hr_patches, data_range=1.0).item()
    
        avg_val_loss = val_loss / len(val_loader)
        avg_val_psnr = val_psnr / len(val_loader)
        avg_val_ssim = val_ssim / len(val_loader)
        writer.add_scalar('Loss/val', avg_val_loss, epoch)
        writer.add_scalar('PSNR/val', avg_val_psnr, epoch)
        writer.add_scalar('SSIM/val', avg_val_ssim, epoch)
        print(f'Validation Loss: {avg_val_loss:.4f}, Validation PSNR: {avg_val_psnr:.4f}, Validation SSIM: {avg_val_ssim:.4f}')

        # Check early stopping condition
        early_stopping(avg_val_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
        # Step the scheduler
        scheduler.step(avg_val_loss)
    
    print('Training complete.')

2024-09-03 07:40:48.212839: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-03 07:40:48.351979: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-03 07:40:49.042439: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-09-03 07:40:49.042508: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

Epoch [20/25], Train Loss: 0.0713, Train PSNR: 26.5379, Train SSIM: 0.8230


Validation Loss: 0.0721, Validation PSNR: 26.4905, Validation SSIM: 0.8205
Validation loss decreased (inf --> 0.072086).  Saving model ...


Epoch 21/25: 100%|██████████| 2363/2363 [1:06:03<00:00,  1.68s/batch, loss=0.079] 

Epoch [21/25], Train Loss: 0.0711, Train PSNR: 26.5909, Train SSIM: 0.8236


...

Epoch [24/25], Train Loss: 0.0702, Train PSNR: 26.7097, Train SSIM: 0.8256


Validation Loss: 0.0717, Validation PSNR: 26.5632, Validation SSIM: 0.8209
Validation loss decreased (0.072086 --> 0.071669).  Saving model ...


Epoch 25/25: 100%|██████████| 2363/2363 [1:06:12<00:00,  1.68s/batch, loss=0.0878]

Epoch [25/25], Train Loss: 0.0700, Train PSNR: 26.6955, Train SSIM: 0.8260


Validation Loss: 0.0725, Validation PSNR: 26.2958, Validation SSIM: 0.8188
EarlyStopping counter: 1 out of 5
Training complete.


In [15]:
torch.save(model, "UNet_l1_ssim_final.pth")

### Training the model for Charbonnier loss

In [ ]:
from torch.cuda.amp import GradScaler, autocast

torch.autograd.set_detect_anomaly(True)

if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = UNetSRx4RiR(in_channels=3, out_channels=3, num_features=64, dropout_rate=0.0).to(device)
    criterion = CharbonnierLoss(epsilon=1e-4)
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    scaler = GradScaler()
    
    # Initialize TensorBoard writer
    from torch.utils.tensorboard import SummaryWriter
    writer = SummaryWriter('runs/UNetSRx4_charbonnier')

    # Initialize early stopping
    early_stopping = EarlyStopping(patience=5, verbose=True)
    
    # Initialize learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=2, verbose=True)

    num_epochs = 25
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_psnr = 0.0
        running_ssim = 0.0

        # Wrap the train_loader with tqdm for progress bar
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
        
        for i, (hr_patches, lr_patches) in enumerate(progress_bar):
            hr_patches, lr_patches = hr_patches.to(device), lr_patches.to(device)
            
            optimizer.zero_grad()
            with autocast(enabled=True):
                outputs = model(lr_patches)
                loss = criterion(outputs, hr_patches)
                if torch.isnan(loss).any():
                    print("Loss is NaN")
                    break
            
            scaler.scale(loss).backward()
            # Apply gradient clipping
            # nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
            
            running_loss += loss.item()
            running_psnr += calculate_psnr(outputs, hr_patches).item()
            outputs = outputs.float()
            hr_patches = hr_patches.float()
            running_ssim += ssim(outputs, hr_patches, data_range=1.0).item()

            # Update the progress bar with the running loss
            progress_bar.set_postfix(loss=loss.item())
        
        avg_train_loss = running_loss / len(train_loader)
        avg_train_psnr = running_psnr / len(train_loader)
        avg_train_ssim = running_ssim / len(train_loader)
        writer.add_scalar('Loss/train', avg_train_loss, epoch)
        writer.add_scalar('PSNR/train', avg_train_psnr, epoch)
        writer.add_scalar('SSIM/train', avg_train_ssim, epoch)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train PSNR: {avg_train_psnr:.4f}, Train SSIM: {avg_train_ssim:.4f}')
    
        # Validation loop
        model.eval()
        val_loss = 0.0
        val_psnr = 0.0
        val_ssim = 0.0     
        with torch.no_grad():
            for hr_patches, lr_patches in val_loader:
                hr_patches, lr_patches = hr_patches.to(device), lr_patches.to(device)
                with autocast(enabled=True):
                    outputs = model(lr_patches)
                    loss = criterion(outputs, hr_patches)
                    if torch.isnan(loss).any():
                        print("Loss is NaN")
                        break
                val_loss += loss.item()
                val_psnr += calculate_psnr(outputs, hr_patches).item()
                # Convert to float32 before calculating SSIM
                outputs = outputs.float()
                hr_patches = hr_patches.float()
                val_ssim += ssim(outputs, hr_patches, data_range=1.0).item()
    
        avg_val_loss = val_loss / len(val_loader)
        avg_val_psnr = val_psnr / len(val_loader)
        avg_val_ssim = val_ssim / len(val_loader)
        writer.add_scalar('Loss/val', avg_val_loss, epoch)
        writer.add_scalar('PSNR/val', avg_val_psnr, epoch)
        writer.add_scalar('SSIM/val', avg_val_ssim, epoch)
        print(f'Validation Loss: {avg_val_loss:.4f}, Validation PSNR: {avg_val_psnr:.4f}, Validation SSIM: {avg_val_ssim:.4f}')

        # Check early stopping condition
        early_stopping(avg_val_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
        # Step the scheduler
        scheduler.step(avg_val_loss)
    
    print('Training complete.')

2024-09-04 04:24:43.462607: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-04 04:24:43.603426: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-04 04:24:44.318968: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-09-04 04:24:44.319057: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

Epoch [1/25], Train Loss: 0.0455, Train PSNR: 23.6736, Train SSIM: 0.7262


Validation Loss: 0.0327, Validation PSNR: 25.4157, Validation SSIM: 0.7709
Validation loss decreased (inf --> 0.032664).  Saving model ...


Epoch 2/25: 100%|██████████| 1182/1182 [23:26<00:00,  1.19s/batch, loss=0.0362]

Epoch [2/25], Train Loss: 0.0338, Train PSNR: 25.2284, Train SSIM: 0.7751


...

Validation Loss: 0.0255, Validation PSNR: 26.5442, Validation SSIM: 0.8080
Validation loss decreased (0.026546 --> 0.025503).  Saving model ...


Epoch 16/25: 100%|██████████| 1182/1182 [23:31<00:00,  1.19s/batch, loss=0.0365]

Epoch [16/25], Train Loss: 0.0260, Train PSNR: 26.3464, Train SSIM: 0.8107


Validation Loss: 0.0254, Validation PSNR: 26.3714, Validation SSIM: 0.8083
Validation loss decreased (0.025503 --> 0.025384).  Saving model ...


Epoch 17/25:  57%|█████▋    | 678/1182 [13:29<10:02,  1.19s/batch, loss=0.0228]

Continuing the training after epoch 16 as the session abrutly ended

In [13]:
# Continuing the training after epoch 16 as the session abrutly ended
from torch.cuda.amp import GradScaler, autocast

torch.autograd.set_detect_anomaly(True)

if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = UNetSRx4RiR(in_channels=3, out_channels=3, num_features=64, dropout_rate=0.0).to(device)
    criterion = CharbonnierLoss(epsilon=1e-4)
    
    # Load the model's state dict from the checkpoint
    checkpoint = torch.load('checkpoint.pt')
    model.load_state_dict(checkpoint)
    
    optimizer = optim.Adam(model.parameters(), lr=0.00005) # Reducing the loss function slightly
    scaler = GradScaler()
    start_epoch = 16  # Start from the next epoch
    
    # Initialize TensorBoard writer
    from torch.utils.tensorboard import SummaryWriter
    writer = SummaryWriter('runs/UNetSRx4_charbonnier')

    # Initialize early stopping
    early_stopping = EarlyStopping(patience=5, verbose=True)
    
    # Initialize learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=2, verbose=True)

    num_epochs = 25
    for epoch in range(start_epoch, num_epochs): # Start from start_rpoch
        model.train()
        running_loss = 0.0
        running_psnr = 0.0
        running_ssim = 0.0

        # Wrap the train_loader with tqdm for progress bar
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
        
        for i, (hr_patches, lr_patches) in enumerate(progress_bar):
            hr_patches, lr_patches = hr_patches.to(device), lr_patches.to(device)
            
            optimizer.zero_grad()
            with autocast(enabled=True):
                outputs = model(lr_patches)
                loss = criterion(outputs, hr_patches)
                if torch.isnan(loss).any():
                    print("Loss is NaN")
                    break
            
            scaler.scale(loss).backward()
            # Apply gradient clipping
            # nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
            
            running_loss += loss.item()
            running_psnr += calculate_psnr(outputs, hr_patches).item()
            outputs = outputs.float()
            hr_patches = hr_patches.float()
            running_ssim += ssim(outputs, hr_patches, data_range=1.0).item()

            # Update the progress bar with the running loss
            progress_bar.set_postfix(loss=loss.item())
        
        avg_train_loss = running_loss / len(train_loader)
        avg_train_psnr = running_psnr / len(train_loader)
        avg_train_ssim = running_ssim / len(train_loader)
        writer.add_scalar('Loss/train', avg_train_loss, epoch)
        writer.add_scalar('PSNR/train', avg_train_psnr, epoch)
        writer.add_scalar('SSIM/train', avg_train_ssim, epoch)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train PSNR: {avg_train_psnr:.4f}, Train SSIM: {avg_train_ssim:.4f}')
    
        # Validation loop
        model.eval()
        val_loss = 0.0
        val_psnr = 0.0
        val_ssim = 0.0     
        with torch.no_grad():
            for hr_patches, lr_patches in val_loader:
                hr_patches, lr_patches = hr_patches.to(device), lr_patches.to(device)
                with autocast(enabled=True):
                    outputs = model(lr_patches)
                    loss = criterion(outputs, hr_patches)
                    if torch.isnan(loss).any():
                        print("Loss is NaN")
                        break
                val_loss += loss.item()
                val_psnr += calculate_psnr(outputs, hr_patches).item()
                # Convert to float32 before calculating SSIM
                outputs = outputs.float()
                hr_patches = hr_patches.float()
                val_ssim += ssim(outputs, hr_patches, data_range=1.0).item()
    
        avg_val_loss = val_loss / len(val_loader)
        avg_val_psnr = val_psnr / len(val_loader)
        avg_val_ssim = val_ssim / len(val_loader)
        writer.add_scalar('Loss/val', avg_val_loss, epoch)
        writer.add_scalar('PSNR/val', avg_val_psnr, epoch)
        writer.add_scalar('SSIM/val', avg_val_ssim, epoch)
        print(f'Validation Loss: {avg_val_loss:.4f}, Validation PSNR: {avg_val_psnr:.4f}, Validation SSIM: {avg_val_ssim:.4f}')

        # Check early stopping condition
        early_stopping(avg_val_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
        # Step the scheduler
        scheduler.step(avg_val_loss)
    
    print('Training complete.')

2024-09-04 15:36:39.383257: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-04 15:36:49.704962: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-04 15:37:36.680281: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-09-04 15:37:36.680679: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

Epoch [17/25], Train Loss: 0.0256, Train PSNR: 26.4188, Train SSIM: 0.8120


Validation Loss: 0.0254, Validation PSNR: 26.5091, Validation SSIM: 0.8177
Validation loss decreased (inf --> 0.025369).  Saving model ...


Epoch 18/25: 100%|██████████| 1182/1182 [23:10<00:00,  1.18s/batch, loss=0.0415]

Epoch [18/25], Train Loss: 0.0256, Train PSNR: 26.4330, Train SSIM: 0.8123


...

Epoch [24/25], Train Loss: 0.0252, Train PSNR: 26.4916, Train SSIM: 0.8146


Validation Loss: 0.0249, Validation PSNR: 26.5465, Validation SSIM: 0.8187
Validation loss decreased (0.025020 --> 0.024882).  Saving model ...


Epoch 25/25: 100%|██████████| 1182/1182 [23:11<00:00,  1.18s/batch, loss=0.0185]

Epoch [25/25], Train Loss: 0.0252, Train PSNR: 26.5254, Train SSIM: 0.8150


Validation Loss: 0.0256, Validation PSNR: 26.3645, Validation SSIM: 0.8166
EarlyStopping counter: 1 out of 5
Training complete.


In [14]:
torch.save(model, "UNet_charbonnier_eps1e-4_final.pth")

### Training the model for TV + MAE loss

In [14]:
from torch.cuda.amp import GradScaler, autocast

torch.autograd.set_detect_anomaly(True)

if __name__ == "__main__":

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = UNetSRx4RiR(in_channels=3, out_channels=3, num_features=64, dropout_rate=0.0).to(device)
    l1_loss = nn.L1Loss()
    tv_loss = TVLoss(tv_weight=1e-4)
    optimizer = optim.Adam(model.parameters(), lr=0.00005)
    scaler = GradScaler()
    
    # Initialize TensorBoard writer
    from torch.utils.tensorboard import SummaryWriter
    writer = SummaryWriter('runs/UNetSRx4_tv-1e-4_mae_final')

    # Initialize early stopping
    early_stopping = EarlyStopping(patience=5, verbose=True)
    
    # Initialize learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=2, verbose=True)

    num_epochs = 25
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_psnr = 0.0
        running_ssim = 0.0

        # Wrap the train_loader with tqdm for progress bar
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
        
        for i, (hr_patches, lr_patches) in enumerate(progress_bar):
            hr_patches, lr_patches = hr_patches.to(device), lr_patches.to(device)
            
            optimizer.zero_grad()
            with autocast(enabled=True):
                outputs = model(lr_patches)
                loss = combined_tv_mae_loss(outputs, hr_patches)
                if torch.isnan(loss).any():
                    print("Loss is NaN")
                    continue
            
            scaler.scale(loss).backward()
            # Apply gradient clipping
            # nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
            
            running_loss += loss.item()
            running_psnr += calculate_psnr(outputs, hr_patches).item()
            outputs = outputs.float()
            hr_patches = hr_patches.float()
            running_ssim += ssim(outputs, hr_patches, data_range=1.0).item()

            # Update the progress bar with the running loss
            progress_bar.set_postfix(loss=loss.item())
        
        avg_train_loss = running_loss / len(train_loader)
        avg_train_psnr = running_psnr / len(train_loader)
        avg_train_ssim = running_ssim / len(train_loader)
        writer.add_scalar('Loss/train', avg_train_loss, epoch)
        writer.add_scalar('PSNR/train', avg_train_psnr, epoch)
        writer.add_scalar('SSIM/train', avg_train_ssim, epoch)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train PSNR: {avg_train_psnr:.4f}, Train SSIM: {avg_train_ssim:.4f}')
    
        # Validation loop
        model.eval()
        val_loss = 0.0
        val_psnr = 0.0
        val_ssim = 0.0     
        with torch.no_grad():
            for hr_patches, lr_patches in val_loader:
                hr_patches, lr_patches = hr_patches.to(device), lr_patches.to(device)
                with autocast(enabled=True):
                    outputs = model(lr_patches)
                    loss = combined_tv_mae_loss(outputs, hr_patches)
                    if torch.isnan(loss).any():
                        print("Loss is NaN")
                        continue
                val_loss += loss.item()
                val_psnr += calculate_psnr(outputs, hr_patches).item()
                # Convert to float32 before calculating SSIM
                outputs = outputs.float()
                hr_patches = hr_patches.float()
                val_ssim += ssim(outputs, hr_patches, data_range=1.0).item()
    
        avg_val_loss = val_loss / len(val_loader)
        avg_val_psnr = val_psnr / len(val_loader)
        avg_val_ssim = val_ssim / len(val_loader)
        writer.add_scalar('Loss/val', avg_val_loss, epoch)
        writer.add_scalar('PSNR/val', avg_val_psnr, epoch)
        writer.add_scalar('SSIM/val', avg_val_ssim, epoch)
        print(f'Validation Loss: {avg_val_loss:.4f}, Validation PSNR: {avg_val_psnr:.4f}, Validation SSIM: {avg_val_ssim:.4f}')

        # Check early stopping condition
        early_stopping(avg_val_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
        # Step the scheduler
        scheduler.step(avg_val_loss)
    
    print('Training complete.')

2024-09-06 12:18:04.038916: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-06 12:18:17.010591: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-06 12:19:00.370499: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-09-06 12:19:00.370912: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

Epoch [1/25], Train Loss: 0.0469, Train PSNR: 23.4509, Train SSIM: 0.7100


Validation Loss: 0.0376, Validation PSNR: 24.7016, Validation SSIM: 0.7602
Validation loss decreased (inf --> 0.037649).  Saving model ...


Epoch 2/25: 100%|██████████| 1182/1182 [23:48<00:00,  1.21s/batch, loss=0.0429]

Epoch [2/25], Train Loss: 0.0353, Train PSNR: 24.9652, Train SSIM: 0.7632


...

Epoch [24/25], Train Loss: 0.0253, Train PSNR: 26.4742, Train SSIM: 0.8126


Validation Loss: 0.0254, Validation PSNR: 26.4898, Validation SSIM: 0.8113
EarlyStopping counter: 1 out of 5


Epoch 25/25: 100%|██████████| 1182/1182 [23:44<00:00,  1.21s/batch, loss=0.0176]

Epoch [25/25], Train Loss: 0.0253, Train PSNR: 26.5016, Train SSIM: 0.8130


Validation Loss: 0.0252, Validation PSNR: 26.5008, Validation SSIM: 0.8117
Validation loss decreased (0.025232 --> 0.025196).  Saving model ...
Training complete.


In [15]:
torch.save(model, "UNet_1e-4tv_mae_final.pth")